In [1]:
# !pip install xgboost imbalanced-learn tensorflow==2.15 --quiet

import numpy as np, pandas as pd, joblib, math, random, gc, warnings, os, xgboost as xgb
from pathlib import Path
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import balanced_accuracy_score, recall_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, Model

warnings.filterwarnings("ignore")
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

DATA_DIR   = Path(".")
HR_FILE    = DATA_DIR / "heartrate_15min.csv"
STEPS_FILE = DATA_DIR / "minuteStepsNarrow.csv"
DX_FILE    = DATA_DIR / "Diagnoses_20250404.csv"


In [2]:
diag = (pd.read_csv(DX_FILE, parse_dates=["DCDate.diagnosis_baseline"])
          .rename(columns={"DCDate.diagnosis_baseline": "BaselineDate"})
          .dropna(subset=["BaselineDate"])
          [["PIDN", "BaselineDate", "Diagnosis_baseline_3groups"]])

def slice_days(df, base_date, n=14):
    after = df[df.Time.dt.date >= base_date]
    start = after.Time.min() if not after.empty else df.Time.min()
    end   = start + pd.Timedelta(days=n)
    return df[(df.Time >= start) & (df.Time < end)]


In [3]:
# ---------- Heart-rate stats ----------
hr = pd.read_csv(HR_FILE, parse_dates=["Time"]).merge(diag, "inner", "PIDN")
hr14 = pd.concat([slice_days(g, g.BaselineDate.iloc[0].date(), 14)
                  for _, g in hr.groupby("PIDN")])

def hr_stats(df):
    v = df.Value.to_numpy()
    h = df.Time.dt.hour
    day, night = h.between(6, 21), ~h.between(6, 21)
    rmssd = np.sqrt(np.mean(np.diff(v)**2)) if v.size > 1 else np.nan
    sdnn  = np.std(v, ddof=0)
    return pd.Series({
        "hr_mean": v.mean(), "hr_std": sdnn, "hr_min": v.min(), "hr_max": v.max(),
        "rmssd": rmssd, "lfhf": rmssd/(sdnn+1e-6),
        "day_mean": df.Value[day].mean(), "night_mean": df.Value[night].mean()
    })
feat_hr = hr14.groupby("PIDN").apply(hr_stats).reset_index()

# ---------- Step stats ----------
steps_raw = (pd.read_csv(STEPS_FILE, parse_dates=["ActivityMinute"])
               .rename(columns={"ActivityMinute": "Time", "Steps": "Value"})
               .merge(diag[["PIDN", "BaselineDate"]], "inner"))
step14 = pd.concat([slice_days(g, g.BaselineDate.iloc[0].date(), 14)
                    for _, g in steps_raw.groupby("PIDN")])

step_daily = (step14.assign(Date=step14.Time.dt.date)
                      .groupby(["PIDN", "Date"]).Value.sum()
                      .rename("steps").reset_index())

def st_stats(df):
    v = df.steps.to_numpy()
    trend = v[-1] - v[0] if len(v) > 1 else np.nan
    wknd_mask = pd.to_datetime(df.Date).dt.dayofweek >= 5
    return pd.Series({
        "steps_mean": v.mean(), "steps_std": v.std(ddof=0),
        "steps_min": v.min(), "steps_max": v.max(),
        "steps_trend": trend,
        "wknd_ratio": v[wknd_mask].mean()/(v[~wknd_mask].mean()+1e-6)
    })
feat_st = step_daily.groupby("PIDN").apply(st_stats).reset_index()

# ---------- HR–Steps coupling ----------
hr_daily = (hr14.assign(Date=hr14.Time.dt.date)
                 .groupby(["PIDN", "Date"]).Value.mean()
                 .rename("hr").reset_index())
coupling = (hr_daily.merge(step_daily, "inner")
                   .groupby("PIDN")
                   .apply(lambda d: pd.Series({"hr_steps_corr":
                         np.corrcoef(d.hr, d.steps)[0, 1] if len(d) > 2 else np.nan}))
                   .reset_index())

# ---------- Merge ----------
tab = (feat_hr.merge(feat_st, "left")
              .merge(coupling, "left")
              .merge(diag[["PIDN", "Diagnosis_baseline_3groups"]], "inner"))
X = tab.drop(columns=["Diagnosis_baseline_3groups", "PIDN"])
y = (tab.Diagnosis_baseline_3groups != "Clinically Normal").astype(int).to_numpy()
print("Feature table:", tab.shape)


Feature table: (192, 17)


In [4]:
params_xgb = dict(
    max_depth=4, learning_rate=0.07,
    subsample=0.8, colsample_bytree=0.8,
    objective="binary:logistic", eval_metric="logloss",
    random_state=RANDOM_STATE
)
kf = StratifiedKFold(10, shuffle=True, random_state=RANDOM_STATE)
p_tab = np.zeros_like(y, dtype=float)

for tr, vl in kf.split(X, y):
    bst = xgb.train(params_xgb,
                    xgb.DMatrix(X.iloc[tr], label=y[tr]),
                    num_boost_round=200, verbose_eval=False)
    p_tab[vl] = bst.predict(xgb.DMatrix(X.iloc[vl]))

print("XGB 10-fold OOF BA:",
      balanced_accuracy_score(y, (p_tab >= .5).astype(int)).round(3))
joblib.dump(bst, "models/fitbit_xgb.joblib")


XGB 10-fold OOF BA: 0.534


['models/fitbit_xgb.joblib']

In [6]:
def to_channel(df, full_index, sentinel=-1000):
    s = df.reindex(full_index)["Value"].astype(float)
    mu, sd = s.mean(), s.std(ddof=0)
    return s.sub(mu).div(sd+1e-6).fillna(sentinel).to_numpy()

sentinel = -1000.0
seqs = []
for pid in tab.PIDN:
    idx = hr14.Time[hr14.PIDN == pid].min().floor("D")
    full_idx = pd.date_range(idx, periods=96*14, freq="15min")
    ch_hr = to_channel(hr14[hr14.PIDN == pid].set_index("Time"), full_idx, sentinel)
    st_15 = (step14[step14.PIDN == pid].set_index("Time").Value
               .resample("15min").sum().to_frame("Value"))
    ch_st = to_channel(st_15, full_idx, sentinel)
    seqs.append(np.stack([ch_hr, ch_st], axis=-1))
X_seq = np.stack(seqs)
np.save("seq.npy", X_seq); np.save("labels.npy", y)
print("Sequence tensor", X_seq.shape)


Sequence tensor (192, 1344, 2)


In [7]:
def build_seq_model(input_shape, sentinel=-1000):
    inp = layers.Input(shape=input_shape)
    x = layers.Masking(mask_value=sentinel)(inp)
    x = layers.Bidirectional(layers.GRU(32, return_sequences=True,
                                        dropout=0.2, recurrent_dropout=0.2))(x)
    x = layers.MultiHeadAttention(2, 16)(x, x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.5)(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    mdl = Model(inp, out)
    mdl.compile(tf.keras.optimizers.Adam(1e-3), loss="binary_crossentropy")
    return mdl

p_seq = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(10, shuffle=True, random_state=RANDOM_STATE)

for tr, vl in kf.split(X_seq, y):
    mdl = build_seq_model((1344, 2))
    cw = compute_class_weight("balanced", classes=np.array([0, 1]), y=y[tr])
    mdl.fit(X_seq[tr], y[tr],
            epochs=50, batch_size=32, verbose=0,
            validation_data=(X_seq[vl], y[vl]),
            class_weight={0: cw[0], 1: cw[1]},
            callbacks=[tf.keras.callbacks.EarlyStopping(
                patience=8, restore_best_weights=True, monitor="val_loss")])
    p_seq[vl] = mdl.predict(X_seq[vl], verbose=0).ravel()

print("GRU 10-fold OOF BA:",
      balanced_accuracy_score(y, (p_seq >= .5).astype(int)).round(3))
mdl.save("models/fitbit_gru.h5")


GRU 10-fold OOF BA: 0.658


In [8]:
from sklearn.linear_model import LogisticRegression
meta = LogisticRegression(max_iter=1000, class_weight="balanced")
meta_X = np.column_stack([p_tab, p_seq])
meta.fit(meta_X, y)
print("Blender 10-fold OOF BA:",
      balanced_accuracy_score(y, meta.predict(meta_X)).round(3))
joblib.dump(meta, "models/blender.joblib")


Blender 10-fold OOF BA: 0.677


['models/blender.joblib']

In [11]:
# -------- train/test split ----------
X_tab_tr, X_tab_te, y_tr, y_te, X_seq_tr, X_seq_te = train_test_split(
    X, y, X_seq, test_size=0.20, stratify=y, random_state=RANDOM_STATE)

# -------- retrain XGB ---------------
bst_final = xgb.train(params_xgb,
                      xgb.DMatrix(X_tab_tr, label=y_tr),
                      num_boost_round=200)
probs_tab = bst_final.predict(xgb.DMatrix(X_tab_te))

# -------- retrain GRU ---------------
gru_final = build_seq_model((1344, 2))
cw = compute_class_weight("balanced", classes=np.array([0, 1]), y=y_tr)
gru_final.fit(X_seq_tr, y_tr, epochs=50, batch_size=32, verbose=0,
              validation_split=0.15,
              class_weight={0: cw[0], 1: cw[1]},
              callbacks=[tf.keras.callbacks.EarlyStopping(
                  patience=8, restore_best_weights=True, monitor="val_loss")])
probs_seq = gru_final.predict(X_seq_te, verbose=0).ravel()

# -------- blend ----------------------
probs_blend = meta.predict_proba(np.column_stack([probs_tab, probs_seq]))[:, 1]
y_pred50 = (probs_blend >= 0.50).astype(int)

print("TEST BA (cut 0.50):",
      round(balanced_accuracy_score(y_te, y_pred50), 3))
print("Abnormal recall:",
      round(recall_score(y_te, y_pred50), 3))
print("Confusion:\n", confusion_matrix(y_te, y_pred50))


# -------- bootstrap error bars -------
def bootstrap(y_true, prob, cut, n_iter=1000, frac=0.70):
    rng = np.random.default_rng(RANDOM_STATE)
    n = len(y_true); k = int(math.ceil(frac * n))
    idx = np.arange(n)
    ba, rec = [], []
    for _ in range(n_iter):
        s = rng.choice(idx, k, replace=True)
        pred = (prob[s] >= cut).astype(int)
        ba.append(balanced_accuracy_score(y_true[s], pred))
        rec.append(recall_score(y_true[s], pred))
    return np.mean(ba), np.std(ba), np.mean(rec), np.std(rec)

ba_m, ba_sd, re_m, re_sd = bootstrap(y_te, probs_blend, cut=0.50)
print(f"Bootstrapped (cut 0.50) BA = {ba_m:.3f} ± {ba_sd:.3f}")
print(f"Bootstrapped recall      = {re_m:.3f} ± {re_sd:.3f}")


TEST BA (cut 0.50): 0.577
Abnormal recall: 0.714
Confusion:
 [[11 14]
 [ 4 10]]
Bootstrapped (cut 0.50) BA = 0.575 ± 0.095
Bootstrapped recall      = 0.711 ± 0.148


In [12]:
scan = np.arange(0.35, 0.61, 0.01)
rows = []
for c in scan:
    yp = (probs_blend >= c).astype(int)
    rows.append([c,
                 balanced_accuracy_score(y_te, yp),
                 recall_score(y_te, yp),
                 ((yp==1)&(y_te==0)).sum(),
                 ((yp==0)&(y_te==1)).sum()])
thr_df = pd.DataFrame(rows, columns=["cut", "bal_acc", "abn_recall", "FP", "FN"])
display(thr_df.style.format({"bal_acc": "{:.3f}", "abn_recall": "{:.3f}"}))
best = thr_df.loc[thr_df.bal_acc.idxmax()]
print("Best BA row:\n", best.to_dict())


,cut,bal_acc,abn_recall,FP,FN
0,0.350000,0.613,0.786,14,3
1,0.360000,0.613,0.786,14,3
2,0.370000,0.613,0.786,14,3
3,0.380000,0.613,0.786,14,3
4,0.390000,0.613,0.786,14,3
5,0.400000,0.613,0.786,14,3
6,0.410000,0.613,0.786,14,3
7,0.420000,0.613,0.786,14,3
8,0.430000,0.613,0.786,14,3
9,0.440000,0.613,0.786,14,3


Best BA row:
 {'cut': 0.5600000000000002, 'bal_acc': 0.6171428571428572, 'abn_recall': 0.7142857142857143, 'FP': 12.0, 'FN': 4.0}
